In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-16 18:21:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.21MB/s    in 0.2s    

2022-10-16 18:21:44 (6.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
# load the data file into Spark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...|2005-10-14 00:00:00|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...| 

In [5]:
# all vine reviews, no extra filters
all_vine_reviews_df = df.filter(df.vine == "Y")
all_vine_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   42321535|R2NKNL4WXWIWBG|1400062217|     791531418|The Laments: A Novel|           Books|          4|            3|          3|   Y|                Y|Doh! He *almost* ...|There were times ...|2005-09-12 00:00:00|
|         US|   52254603| RAIG9ZQDMBF5P|0679742034|     586032452|    Buffalo Soldiers| 

In [6]:
# Create the vine_table. [df]
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine", "verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ58W7SMO911M|          4|            2|          3|   N|                N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|                N|
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
| RATOTLA3OF70O|          5|            2|          2|   N|                N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|                N|
|R2F53LI9KK9MOY|          4|            2|          2|   N|                N|
|R1KJ6MB7MRSQFF|          4|            9|         11|   N|                N|
|R2XIM9LT335WHE|          4|            3|         10|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1VKEE2NWSWDRU|          4|            0|          0|   N|     

In [7]:
# filter reviews that have more than 20 total votes
total_votes_df = vine_df.filter(vine_df.total_votes >= 20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1DOSHH6AI622S|          4|            1|         22|   N|                N|
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R1AABFZH0J0C0C|          3|           17|         35|   N|                N|
|R3SJYLG07EHNE9|          2|           16|         34|   N|                N|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
|R1KPOIT5WT3NEF|          1|           13|         29|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
| R33IFUWCNYFSO|          2|            4|         45|   N|                N|
| RWMT2MXG3WLDP|          5|            6|         20|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|     

In [8]:
# filter to show the number of helpful_votes divided by total_votes is equal to or greater than 50%.
filtered_df = total_votes_df.filter((total_votes_df.helpful_votes/total_votes_df.total_votes) >= .5)
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

# create two dfs - vine, non-vine

In [9]:
# vine reviews
vine_reviews_df = total_votes_df.filter(total_votes_df.vine == "Y")
vine_reviews_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [10]:
# non-vine reviews
nonvine_reviews_df = filtered_df.filter(filtered_df.vine == "N")
nonvine_reviews_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VE0FQQ0QTQJN|          5|           16|         20|   N|                Y|
|R3MD5TZJGIPTDU|          5|           21|         31|   N|                N|
| RX6OWUF8EFZ72|          5|           27|         28|   N|                N|
|R1OPUF771LL5O5|          4|           27|         27|   N|                N|
|R3AW5JRZYH66M5|          1|          268|        307|   N|                N|
|R2DXLNKF0EL4CI|          4|           24|         27|   N|                Y|
|R12EHXBC0GP2C2|          5|           23|         24|   N|                N|
|R2JSWNTAO83X3V|          5|          145|        184|   N|                N|
|R3THZWOBS1086J|          5|           46|         47|   N|                N|
|R160DHJPJ4B20R|          4|           28|         30|   N|     

# how many reviews 
* total
* vine
* nonvine

In [13]:
# Total Reviews
total_reviews = filtered_df.count()
total_reviews

403807

In [14]:
# Total Vine
vine_reviews_ct = filtered_df.filter("vine == 'Y'")
vine_reviews_count = vine_reviews_ct.count()
vine_reviews_count

0

In [15]:
# Total NonVine
nonvine_reviews_ct = filtered_df.filter("vine == 'N'")
nonvine_reviews_count = nonvine_reviews_ct.count()
nonvine_reviews_count

403807

# how many reviews are 5-star

In [16]:
# Total 5-star Vine
vine_5star_reviews = vine_reviews_ct.filter("star_rating == 5")
vine_5star_reviews_count = vine_5star_reviews.count()
vine_5star_reviews_count

0

In [17]:
# Total 5-star NonVine
nonvine_5star_reviews = nonvine_reviews_ct.filter("star_rating == 5")
nonvine_5star_reviews_count = nonvine_5star_reviews.count()
nonvine_5star_reviews_count

242889

# what percent of the reviews are 5-star

In [18]:
# % of 5-star Vine
vine_5star_percent = (vine_5star_reviews_count/vine_reviews_count) * 100 
round(vine_5star_percent, 2)

ZeroDivisionError: ignored

In [19]:
# % of 5-star NonVine
nonvine_5star_percent = (nonvine_5star_reviews_count/nonvine_reviews_count) * 100 
round(nonvine_5star_percent, 2)

60.15